In [ ]:
import time
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imutils
import matplotlib.image as mpimg
from collections import OrderedDict
from skimage import io, transform
from math import *
import xml.etree.ElementTree as ET

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import scipy.io
import numpy as np
import glob, os
import pandas as pd
from matplotlib import image
from matplotlib import pyplot as plt
from PIL import Image

torch.backends.cudnn.deterministic = True
random.seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)
torch.manual_seed(0) #setting a seed for same randomness every time

from torch.utils.tensorboard import SummaryWriter

In [ ]:
def getLandmarksFromTxt(txt):
    return np.loadtxt(txt, delimiter = ",") # 44x2 numpy array

In [ ]:
# Reading all the landmarking files and loading them into Python - 11 minutes to run

rootdir = "C:/../Data"

# List all files in the folder - load images, landmarks, bounding box around face
img_dir = os.path.join(rootdir, "imgs")
img_file_list = sorted(os.listdir(img_dir))

lm_dir = os.path.join(rootdir, "lms")
lm_file_list = sorted(os.listdir(lm_dir))

bbox_dir = os.path.join(rootdir, "bbox")
bbox_file_list = sorted(os.listdir(bbox_dir))

num_files = np.shape(img_file_list)[0]
print(num_files)

ids = [] #list of strings
lms = [] #list of numpy arrays
imgs =[] # list of numpy arrays
imgfiles = [] # list of image file paths
crops = [] # list of dictionaries containing cropping details (set to full image right now)

for i in range(num_files): 
    
    #load image
    img_file = rootdir + "/imgs/" + img_file_list[i]
    img = Image.open(img_file) 
    imgarr = np.asarray(img)
    imgfiles.append(img_file)
    imgs.append(imgarr)

    #load lm
    lm_file = rootdir + "/lms/" + lm_file_list[i]
    lm = getLandmarksFromTxt(lm_file)
    lms.append(lm)

    #generate crop
    margin = 10
    allx = lm[:,0] 
    ally = lm[:,1] 
    sx = min(allx)-margin
    sy = min(ally)-margin
    lx = max(allx)+margin
    ly = max(ally)+margin
    w = lx-sx
    h = ly-sy
    cropsDict = {'top': str(int(sy)), 'left' : str(int(sx)), 'width': str(int(w)), 'height' : str(int(h))}
    crops.append(cropsDict)

    #save image id
    id = img_file[0:5] # e.g 09524
    ids.append(id)



# creating a database
data = list(zip(ids, imgs, lms, imgfiles, crops))
df = pd.DataFrame(data, columns=['ID', 'Face Image', 'Landmarks', 'Image Files', 'Crops'])
df        

## Visualize the dataset

In [ ]:
imgnum = 5

landmarks = df['Landmarks'][imgnum]

plt.figure(figsize=(5,5))
plt.imshow(mpimg.imread(df['Image Files'][imgnum]))
plt.scatter(landmarks[:,0], landmarks[:,1], s = 3, c = 'y')
plt.show()

## Create dataset class

In [ ]:
class Transforms():
    def __init__(self):
        pass

    def rotate(self, image, landmarks, angle): # not used.
        angle = random.uniform(-angle, +angle)

        transformation_matrix = torch.tensor([
            [+cos(radians(angle)), -sin(radians(angle))],
            [+sin(radians(angle)), +cos(radians(angle))]
        ])

        image = imutils.rotate(np.array(image), angle)

        #uncomment the two lines below IF you are centering landmarks between -0.5 and 0.5

        #landmarks = landmarks - 0.5  
        new_landmarks = np.matmul(landmarks, transformation_matrix)
        #new_landmarks = new_landmarks + 0.5
        return Image.fromarray(image), new_landmarks

    def resize(self, image, landmarks, img_size):
        image = TF.resize(image, img_size)
        return image, landmarks

    def color_jitter(self, image, landmarks):
        color_jitter = transforms.ColorJitter(brightness=0.3,
                                              contrast=0.3,
                                              saturation=0.3,
                                              hue=0.1)
        image = color_jitter(image)
        return image, landmarks

    def crop_face(self, image, landmarks, crops):
        left = int(crops['left'])
        top = int(crops['top'])
        width = int(crops['width'])
        height = int(crops['height'])

        image = TF.crop(image, top, left, height, width)

        img_shape = np.array(image).shape
        landmarks = torch.tensor(landmarks) - torch.tensor([[left, top]])
        landmarks = landmarks / torch.tensor([img_shape[1], img_shape[0]]) #normalizing the landmarks to be between 0 and 1
        return image, landmarks

    def __call__(self, image, landmarks, crops):
        image = Image.fromarray(image)

        image, landmarks = self.crop_face(image, landmarks, crops) # this is where landmarks also change format
        image, landmarks = self.resize(image, landmarks, (224, 224))
        image, landmarks = self.color_jitter(image, landmarks)
        #image, landmarks = self.rotate(image, landmarks, angle=10)

        image = TF.to_tensor(image)
        image = TF.normalize(image, [0.5], [0.5])
        return image, landmarks

In [ ]:
class FaceLandmarksDataset(Dataset):

    def __init__(self, transform=None, database=None):

        self.image_filenames = database['Image Files'].values
        
        self.landmarks = []
        self.crops = []
        self.transform = transform
        self.landmarks = database['Landmarks'].values 
        
        self.crops = database['Crops'].values

        #print(len(self.image_filenames) ,  len(self.landmarks))
        assert len(self.image_filenames) == len(self.landmarks)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        image = cv2.imread(self.image_filenames[index], 0)
        landmarks = self.landmarks[index]

        if self.transform:
            image, landmarks = self.transform(image, landmarks, self.crops[index])
        
        landmarks = landmarks - 0.5 #centering the data between -0.5 and 0.5
        return image, landmarks


In [ ]:
dataset = FaceLandmarksDataset(transform = Transforms(), database=df)
print(np.shape(dataset[3][0]))


## Visualize Train Transforms

In [ ]:
image, landmarks = dataset[16] #choose an image/lm set

#landmarks = (landmarks) * 224
landmarks = (landmarks + 0.5) * 224
plt.figure(figsize=(5, 5))
plt.imshow(image.numpy().squeeze(), cmap='gray');
plt.scatter(landmarks[:,0], landmarks[:,1], s=3, c='y');

## Split the dataset into train and valid dataset

In [ ]:
# split the dataset into validation and test sets
len_valid_set = int(0.1*len(dataset))
len_test_set = int(0.1*len(dataset))
len_train_set = len(dataset) - len_valid_set - len_test_set

print("The length of Train set is {}".format(len_train_set))
print("The length of Valid set is {}".format(len_valid_set))
print("The length of Test set is {}".format(len_test_set))

torch.manual_seed(0) #setting a seed for same randomness every time
train_dataset , valid_dataset, test_dataset = torch.utils.data.random_split(dataset , [len_train_set, len_valid_set, len_test_set])

# shuffle and batch the datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0) #OG num_workers=4 runs an infinite loop
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=8, shuffle=True, num_workers=0) #OG num_workers=4
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=0) #OG num_workers=4


## Define the model

In [ ]:
class Network(nn.Module):
    def __init__(self,num_classes=88):
        super().__init__()
        self.model_name='resnet18'
        self.model=models.resnet18()
        self.model.conv1=nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc=nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        x=self.model(x)
        return x

## Helper Functions

In [ ]:
import sys

def print_overwrite(step, total_step, loss, operation):
    sys.stdout.write('\r')
    if operation == 'train':
        sys.stdout.write("Train Steps: %d/%d  Loss: %.4f " % (step, total_step, loss))
    else:
        sys.stdout.write("Valid Steps: %d/%d  Loss: %.4f " % (step, total_step, loss))

    sys.stdout.flush()

## Train

In [ ]:
torch.autograd.set_detect_anomaly(True)
#network = Network()
#network.cuda()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network = Network().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(network.parameters(), lr=0.0001)

loss_min = np.inf



In [ ]:
# start a PYTORCH TRAINING ------------

model_path = 'C:/../myModel.pth'
start_time = time.time()
num_epochs = 100

writer = SummaryWriter()

for epoch in range(1,num_epochs+1):
    print("entering epoch: ", epoch)
    loss_train = 0
    loss_valid = 0
    running_loss = 0

    network.train()
    for step in range(1,len(train_loader)+1):

        images, landmarks = next(iter(train_loader))

        #print(landmarks.shape)
        
        images = images.cuda()
        landmarks = landmarks.view(landmarks.size(0),-1).cuda()

        predictions = network(images)
        #print(type(predictions))
        predictions = predictions.to(torch.float64)
        #print(predictions.dtype)

        # clear all the gradients before calculating them
        optimizer.zero_grad()

        # find the loss for the current step
        loss_train_step = criterion(predictions, landmarks)

        # calculate the gradients
        loss_train_step.backward()

        # update the parameters
        optimizer.step()

        loss_train += loss_train_step.item()
        running_loss = loss_train/step

        print_overwrite(step, len(train_loader), running_loss, 'train')

        epoch_len = len(train_loader)
        writer.add_scalar("train_loss", loss_train_step.item(), epoch_len * epoch + step)
    
    network.eval()
    with torch.no_grad():

        for step in range(1,len(valid_loader)+1):

            images, landmarks = next(iter(valid_loader))

            images = images.cuda()
            landmarks = landmarks.view(landmarks.size(0),-1).cuda()

            predictions = network(images)

            # find the loss for the current step
            loss_valid_step = criterion(predictions, landmarks)

            loss_valid += loss_valid_step.item()
            running_loss = loss_valid/step

            print_overwrite(step, len(valid_loader), running_loss, 'valid')

    loss_train /= len(train_loader)
    loss_valid /= len(valid_loader)

    #print('\n--------------------------------------------------')
    print('Epoch: {}  Train Loss: {:.4f}  Valid Loss: {:.4f}'.format(epoch, loss_train, loss_valid))
    #print('--------------------------------------------------')

    if loss_valid < loss_min:
        loss_min = loss_valid
        torch.save(network.state_dict(), model_path)
        print("\nMinimum Validation Loss of {:.4f} at epoch {}/{}".format(loss_min, epoch, num_epochs))
        print('Model Saved\n')
    
    writer.add_scalar("val_mean_dice", loss_valid, epoch + 1)

writer.close()
print('Training Complete')
print("Total Elapsed Time : {} s".format(time.time()-start_time))

## Predict on test Images

In [ ]:
# Function to calculate Euclidean distance between two points
def euclidean_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

# Function to calculate Mean Euclidean Distance Error and Normalized Mean Error
def calculate_errors(ground_truth_landmarks, predicted_landmarks):
    # ground_truth_landmarks is (704, 44, 2)
    num_images = len(ground_truth_landmarks)
    num_landmarks = len(ground_truth_landmarks[0])  # Assuming the same number of landmarks for all images

    MED = []
    NME_IP = []
    NME_IO = []
    MNE = []

    for i in range(num_images):
        
        interpupilar_distance = euclidean_distance(ground_truth_landmarks[i][4], ground_truth_landmarks[i][9]) #5th & 10th landmarks are eye pupils
        interoccular_distance = euclidean_distance(ground_truth_landmarks[i][0], ground_truth_landmarks[i][7]) #1st & 8th landmarks are outer corners of the eye

        #print("eye pupil locations: ", ground_truth_landmarks[i][4], ground_truth_landmarks[i][9])
        #print("ID = ", interpupilar_distance)
        
        ed_img = 0
        ed_img_normIP = 0
        ed_img_normIO = 0
        norm_error = 0
        fr = 0

        for j in range(num_landmarks):
            ed_lm = euclidean_distance(predicted_landmarks[i][j], ground_truth_landmarks[i][j])
            ed_lm_normIP = ed_lm / interpupilar_distance
            ed_lm_normIO = ed_lm / interoccular_distance
            ed_lm_normBB = ed_lm / 224 #bounding box size #Grooby2023
            
            ed_img += ed_lm
            ed_img_normIP += ed_lm_normIP
            ed_img_normIO += ed_lm_normIO
            norm_error += ed_lm_normBB
        
        #print("Mean Euclidean Distance (MED) error of image = ", ed_img/num_landmarks)
        #print("Normalized Mean Error (NME) of image = ", ed_img_norm/num_landmarks)
        #print()
        
        MED.append(round(ed_img/num_landmarks, 3))
        NME_IP.append (round(ed_img_normIP/num_landmarks, 3))
        NME_IO.append (round(ed_img_normIO/num_landmarks, 3))
        MNE.append (round(norm_error/num_landmarks, 3))

    return MED,NME_IP,NME_IO,MNE


In [ ]:
#calculating predicted landmarks in test set
model_path = model_path

ground_truth_landmarks_list = []
predicted_landmarks_list = []
i=0

with torch.no_grad():

    best_network = Network()
    best_network.cuda()
    best_network.load_state_dict(torch.load(model_path))
    best_network.eval()

    x = len(test_loader)+1   ## Think multiples of 8 (x=3 will display 2*8=16 images)

    for step in range(1,x):

        images, landmarks = next(iter(test_loader))
        
        images = images.cuda()
        #landmarks = (landmarks) * 224 #8x44x2
        landmarks = (landmarks + 0.5) * 224 #8x44x2

        #predictions = (best_network(images).cpu() ) * 224
        predictions = (best_network(images).cpu() + 0.5) * 224
        predictions = predictions.view(-1,44,2) #8x44x2

        for img_num in range(len(landmarks)):
            ground_truth_landmarks_list.append( np.array(landmarks[img_num,:,:]) ) 
            predicted_landmarks_list.append( np.array(predictions[img_num,:,:]) )
            i = i+1


In [ ]:
print(np.shape(ground_truth_landmarks_list))
print(np.shape(predicted_landmarks_list))

# print(ground_truth_landmarks_list[13][0:5][:])
# print(predicted_landmarks_list[13][0:5][:])

In [ ]:
#print('Total number of test images: {}'.format(len(test_dataset)))
#print("Total images processed = ",i, "\n")

med, nmeIP, nmeIO, mne = calculate_errors(ground_truth_landmarks_list, predicted_landmarks_list)

print("Number of images processed = ", len(nmeIP))
print(f"Mean Euclidean Distance Error (MED) per image: {med}")
print(f"Normalized Mean Error (NME) normalized by IPD per image: {nmeIP}")
print(f"Normalized Mean Error (NME) normalized by IOD per image: {nmeIO}")
print(f"Mean Normalized Error (MNE) normalized by Bounding Box per image: {mne}")

print(f"Average MED: {round(np.mean(med), 3)}")
print(f"Average NME_IP: {round(np.mean(nmeIP), 3)}")
print(f"Average NME_IO: {round(np.mean(nmeIO), 3)}")
print(f"Average MNE: {round(np.mean(mne), 3)}")

all_threshold = [0.05, 0.10, 0.15]
FR = []
print("Failure rate for the given thresholds is:")
for threshold in all_threshold:
    fails = [(val>threshold) for val in nmeIO ]
    FR.append(( fails.count(True) / len(nmeIP) )*100)
print(np.round(FR,3))

In [ ]:
#saving average results to a text file
# 
fname = os.path.basename(model_path)[:-4] + ".txt"
file = open(fname,'a+')
print("Number of images processed = ", len(nmeIP), file=file)
print(f"Average MED: {round(np.mean(med), 3)}", file=file)
print(f"Average NME_IP: {round(np.mean(nmeIP), 3)}", file=file)
print(f"Average NME_IO: {round(np.mean(nmeIO), 3)}", file=file)
print(f"Average MNE: {round(np.mean(mne), 3)}", file=file)
print("Threshold[5,10,15] using percentage NMEIO : ", np.round(FR,3), file=file)
file.close() # close the file ones the function execution completes.


In [ ]:
##Write per-image results to a file

fname = os.path.basename(model_path)[:-4] + ".xlsx"

with pd.ExcelWriter(fname) as writer:
    x = np.concatenate((np.expand_dims(np.array(med), 1) , np.expand_dims(np.array(nmeIP), 1), np.expand_dims(np.array(nmeIO), 1), np.expand_dims(np.array(mne), 1)), axis=1)
    x = pd.DataFrame(x, columns=['MED', 'NME_IP', 'NME_IO', 'MNE'])
    x.to_excel(writer)#, sheet_name= "p"+str(i+1), index=False)
